In [1]:
import numpy as np

In [2]:
array = []
with open('min15.txt') as file:
    for line in file.readlines():
        string = line.split(',') 
        del string[:2]
        del string[-1]
        array.append(string)
#         print(line)

In [3]:
data = np.asarray(array[1:])
heads = array[0]

In [4]:
heads

['<DATE>', '<TIME>', '<OPEN>', '<HIGH>', '<LOW>', '<CLOSE>']

In [5]:
time = data[:,1]

In [6]:
minutes = np.asarray([element[2:4] for element in time]).reshape(-1,1).astype(np.int32)
hours = np.asarray([element[:2] for element in time]).reshape(-1,1).astype(np.int32)

In [7]:
date = data[:,0]

In [8]:
year = np.asarray([element[:4] for element in date]).reshape(-1,1).astype(np.int32)
month = np.asarray([element[4:6] for element in date]).reshape(-1,1).astype(np.int32)
day = np.asarray([element[6:] for element in date]).reshape(-1,1).astype(np.int32)

In [9]:
data = np.delete(data,1,1)
data = np.delete(data,0,1)
# data = data.astype(np.float32)
data = np.hstack((year,month,day,hours,minutes,data))

In [11]:
data[1]

array(['2015', '11', '2', '0', '0', '63.6750000', '63.8427000',
       '63.6750000', '63.7588000'], dtype='<U11')

In [12]:
train_data = data[:,5]

In [13]:
ds_x = []
ds_y = []
days_hist= 10
days_predict = 10
period = 24 * 4 * days_hist
period_predict = 24 * 4 * days_predict
for num,line in enumerate(train_data[:-1920]):
    ds_x.append(train_data[num:num+960].reshape((train_data[num:num+960].size)))
for line in train_data[period+period_predict:]:
    ds_y.append(line)

In [15]:
ds_x = np.asarray(ds_x)
ds_y = np.asarray(ds_y).reshape(-1,1)

In [16]:
ds_x.shape

(103690, 960)

In [17]:
ds_y.shape

(103690, 1)